In [1]:
# ===== CONFIGURATION BLOCK =====
# Input and output folders
INPUT_FOLDER = "People_Input"
OUTPUT_FOLDER = "People_Output"

# Load environment variables
from dotenv import load_dotenv, find_dotenv
import os


# Load .env and override any existing environment variables so changes are picked up without restarting the kernel
load_dotenv(find_dotenv(), override=True)

REGION = os.getenv('REGION')
API_KEY_FACE = os.getenv('API_KEY_FACE')
URL_FACE = os.getenv('URL_FACE')

print('Using face endpoint:', URL_FACE)
print('Face API key present:', bool(API_KEY_FACE))

Using face endpoint: https://facefrance.cognitiveservices.azure.com/
Face API key present: True


In [4]:
# ===== INSTALLATION AND IMPORTS =====
import os
import shutil
import json
from pathlib import Path
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from PIL import Image
import hashlib

# Initialize Face client
face_client = FaceClient(URL_FACE, CognitiveServicesCredentials(API_KEY_FACE))

print(f"✓ Connected to Azure Face API in {REGION}")
print(f"⚠️  Note: Most face attributes require Limited Access approval")
print(f"   Using only unrestricted features: blur, head pose, mask detection")


✓ Connected to Azure Face API in francecentral
⚠️  Note: Most face attributes require Limited Access approval
   Using only unrestricted features: blur, head pose, mask detection


In [6]:
# ===== FACE DETECTION WITH UNRESTRICTED FEATURES - FIXED =====

def process_photos_unrestricted(input_folder, output_folder):
    """
    Process photos using ONLY unrestricted Face API features.
    No approval required.
    """
    
    # Create output folder
    os.makedirs(output_folder, exist_ok=True)
    
    # Statistics
    stats = {
        'photos_processed': 0,
        'unique_faces': 0,
        'total_faces_detected': 0,
        'face_list': []
    }
    
    # Metadata storage
    metadata = {}
    
    # Face counter
    face_counter = 1
    
    # Get all image files
    image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.gif')
    image_files = [f for f in os.listdir(input_folder) 
                   if f.lower().endswith(image_extensions)]
    
    if not image_files:
        print(f"❌ No images found in {input_folder}")
        return stats
    
    print(f"\n📁 Processing {len(image_files)} images from {input_folder}...\n")
    
    for idx, image_file in enumerate(image_files, 1):
        image_path = os.path.join(input_folder, image_file)
        print(f"[{idx}/{len(image_files)}] Processing: {image_file}")
        
        try:
            # Detect faces with ONLY unrestricted attributes
            # IMPORTANT: Must specify recognition_model when using qualityForRecognition
            with open(image_path, 'rb') as image_stream:
                detected_faces = face_client.face.detect_with_stream(
                    image=image_stream,
                    detection_model='detection_03',
                    recognition_model='recognition_04',  # REQUIRED for qualityForRecognition
                    return_face_id=False,
                    return_face_landmarks=False,
                    return_face_attributes=['headPose', 'mask', 'blur', 'qualityForRecognition']
                )
            
            stats['photos_processed'] += 1
            face_count = len(detected_faces)
            
            if face_count == 0:
                # No faces detected
                print(f"  ℹ️  No faces detected")
                no_face_folder = os.path.join(output_folder, "No_Faces_Detected")
                os.makedirs(no_face_folder, exist_ok=True)
                shutil.copy2(image_path, os.path.join(no_face_folder, image_file))
                
                metadata[image_file] = {
                    'faces_detected': 0,
                    'faces': []
                }
                continue
            
            print(f"  ✓ Detected {face_count} face(s)")
            stats['total_faces_detected'] += face_count
            stats['unique_faces'] += face_count
            
            image_faces = []
            
            # Process each detected face
            for face in detected_faces:
                face_id = f"Face_{face_counter:04d}"
                face_counter += 1
                
                attrs = face.face_attributes
                rect = face.face_rectangle
                
                # Extract available attributes
                head_pose = None
                mask_info = None
                blur_info = None
                quality = None
                
                if attrs:
                    if attrs.head_pose:
                        head_pose = {
                            'pitch': round(attrs.head_pose.pitch, 2),
                            'roll': round(attrs.head_pose.roll, 2),
                            'yaw': round(attrs.head_pose.yaw, 2)
                        }
                    
                    if attrs.mask:
                        mask_info = {
                            'type': attrs.mask.type,
                            'nose_and_mouth_covered': attrs.mask.nose_and_mouth_covered
                        }
                    
                    if attrs.blur:
                        blur_info = {
                            'blur_level': attrs.blur.blur_level,
                            'value': round(attrs.blur.value, 2)
                        }
                    
                    if hasattr(attrs, 'quality_for_recognition'):
                        quality = attrs.quality_for_recognition
                
                # Display info
                info_parts = [face_id]
                if mask_info:
                    mask_status = "Mask" if mask_info['nose_and_mouth_covered'] else "No mask"
                    info_parts.append(mask_status)
                if quality:
                    info_parts.append(f"Quality: {quality}")
                if blur_info:
                    info_parts.append(f"Blur: {blur_info['blur_level']}")
                
                print(f"    → {', '.join(info_parts)}")
                
                # Store face information
                face_info = {
                    'face_id': face_id,
                    'location': {
                        'left': rect.left,
                        'top': rect.top,
                        'width': rect.width,
                        'height': rect.height
                    },
                    'attributes': {
                        'head_pose': head_pose,
                        'mask': mask_info,
                        'blur': blur_info,
                        'quality_for_recognition': quality
                    }
                }
                
                image_faces.append(face_info)
                stats['face_list'].append({
                    'face_id': face_id,
                    'image': image_file
                })
                
                # Create folder and copy image
                folder_name = f"{face_id}"
                if mask_info and mask_info['nose_and_mouth_covered']:
                    folder_name += "_WithMask"
                if quality:
                    folder_name += f"_{quality}"
                
                face_folder = os.path.join(output_folder, folder_name)
                os.makedirs(face_folder, exist_ok=True)
                shutil.copy2(image_path, os.path.join(face_folder, image_file))
            
            # Store metadata
            metadata[image_file] = {
                'faces_detected': face_count,
                'faces': image_faces
            }
            
        except Exception as e:
            print(f"  ✗ Error processing {image_file}: {str(e)}")
            stats['photos_processed'] += 1
    
    # Save metadata
    metadata_path = os.path.join(output_folder, 'face_metadata.json')
    with open(metadata_path, 'w', encoding='utf-8') as f:
        json.dump(metadata, f, indent=2, ensure_ascii=False)
    
    print(f"\n✓ Metadata saved to: {metadata_path}")
    
    # Print summary
    print("\n" + "="*70)
    print("📊 PROCESSING SUMMARY")
    print("="*70)
    print(f"Number of photos processed: {stats['photos_processed']}")
    print(f"Number of unique faces detected: {stats['unique_faces']}")
    print(f"Total number of faces detected: {stats['total_faces_detected']}")
    
    print(f"\n👤 List of detected face IDs:")
    
    if stats['face_list']:
        for face in stats['face_list']:
            print(f"  - {face['face_id']} (from {face['image']})")
    else:
        print("  (No faces detected in any images)")
    
    print("="*70)
    
    print(f"\n⚠️  IMPORTANT NOTE FOR ASSIGNMENT:")
    print(f"   Azure Face API requires Limited Access approval for:")
    print(f"   - Face identification (PersonGroup)")
    print(f"   - Face attributes (age, gender, emotion, etc.)")
    print(f"   - Celebrity detection (deprecated)")
    print(f"\n   This solution uses only UNRESTRICTED features:")
    print(f"   ✓ Face detection (location, count)")
    print(f"   ✓ Head pose (pitch, roll, yaw)")
    print(f"   ✓ Mask detection")
    print(f"   ✓ Image quality metrics")
    print(f"   ✓ Blur detection")
    print(f"\n   Apply for access at: https://aka.ms/facerecognition")
    
    return stats

# Run the processing
stats = process_photos_unrestricted(INPUT_FOLDER, OUTPUT_FOLDER)



📁 Processing 4 images from People_Input...

[1/4] Processing: gonca3.jpg
  ✓ Detected 2 face(s)
    → Face_0001, No mask, Quality: QualityForRecognition.high, Blur: BlurLevel.low
    → Face_0002, No mask, Quality: QualityForRecognition.low, Blur: BlurLevel.medium
[2/4] Processing: mada1.jpg
  ✓ Detected 1 face(s)
    → Face_0003, No mask, Quality: QualityForRecognition.high, Blur: BlurLevel.low
[3/4] Processing: mada2.jpg
  ✓ Detected 1 face(s)
    → Face_0004, No mask, Quality: QualityForRecognition.high, Blur: BlurLevel.low
[4/4] Processing: photo.jpg
  ✓ Detected 1 face(s)
    → Face_0005, No mask, Quality: QualityForRecognition.high, Blur: BlurLevel.low

✓ Metadata saved to: People_Output\face_metadata.json

📊 PROCESSING SUMMARY
Number of photos processed: 4
Number of unique faces detected: 5
Total number of faces detected: 5

👤 List of detected face IDs:
  - Face_0001 (from gonca3.jpg)
  - Face_0002 (from gonca3.jpg)
  - Face_0003 (from mada1.jpg)
  - Face_0004 (from mada2.jpg)
 